In [1]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

import glob
import itertools
import os

from run import Run
from runs_db import db as runs_db

[('martak', 102), ('gcp', 60), ('deepnet2', 59), ('fiit-gcp-1', 7), ('fiit-gcp-2', 7), ('fiit-gcp-3', 7), ('acer', 6)]


In [ ]:
log_path = '/home/fiit/logs'
runs = []

for server in runs_db:
    paths = glob.glob(os.path.join(log_path, server, '*'))
    paths = iter(sorted(paths))

    try:
        for (number, type_, code) in runs_db[server]:
            for _ in range(number):
                try:
                    path = next(paths)
                    runs.append(Run(path, type_, code))
                except KeyError:
                    print(path)
    except StopIteration:
        pass


In [ ]:
tasks = ['dep', 'lmo', 'ner', 'pos']
langs = ['cs', 'de', 'en', 'es']

task_metr = {
    'dep': 'las',
    'lmo': 'perplexity',
    'ner': 'chunk_f1',
    'pos': 'acc'
}

task_max = {
    'dep': True,
    'lmo': False,
    'ner': True,
    'pos': True
}

def draw_graphs(metric_func, tasks, langs, role, run_codes=None, run_types=None):
    fig, axes = plt.subplots(len(tasks), len(langs), figsize=(5*len(langs), 4*len(tasks)), squeeze=False)

    relevant_runs = ['vanilla']

    for task, lang in itertools.product(tasks, langs):
        for run in runs:
            if (
                run.contains(task, lang) and
                (run_codes is None or run.code in run_codes) and
                (run_types is None or run.type in run_types)
            ):

                history = run.history(
                    metric=metric_func(task),
                    task=task,
                    language=lang,
                    role=role)
                axes[tasks.index(task), langs.index(lang)].plot(list(history), label=f'{run.code}-{run.type}')

    for ax, col in zip(axes[0], langs):
        ax.set_title(col)

    for ax, row in zip(axes[:, 0], tasks):
        ax.set_ylabel(row, rotation=0, size='large')
        
    for ax_row in axes:
        for ax in ax_row:
            ax.legend()

    plt.show()
    
def print_results(metric_func, metric_max_func, tasks, langs, run_codes=None, run_types=None):
    
    for task, lang in itertools.product(tasks, langs):
        for run in runs:
            if (
                run.contains(task, lang) and
                (run_codes is None or run.code in run_codes) and
                (run_types is None or run.type in run_types)
            ):
                result = run.metric_eval(
                    metric=metric_func(task),
                    max_=metric_max_func(task),
                    task=task,
                    language=lang)
                res = result[0]
                if res < 1.01:
                    res *= 100
                print(f'{res:.2f} ', end='')
    print()
        

In [ ]:
# draw_graphs(lambda task: task_metr[task], tasks, langs, 'test',
#              ['private-focused-0.75-dep-ner-pos-cs-es'], ['stsl', 'mt', 'ml', 'mtml'])
print('89.16 86.51 98.97 96.87 81.58 87.23')
print('mt')
print_results(lambda task: task_metr[task], lambda task: task_max[task], ['dep', 'pos', 'ner'], langs,
            ['no-adv-tsh-focused-dep-ner-pos-cs-es'], ['mt'])
print_results(lambda task: task_metr[task], lambda task: task_max[task], ['dep', 'pos', 'ner'], langs,
            ['private-focused-dep-ner-pos-cs-es'], ['mt'])
print_results(lambda task: task_metr[task], lambda task: task_max[task], ['dep', 'pos', 'ner'], langs,
            ['no-adv-tsh-focused-0.75-dep-ner-pos-cs-es'], ['mt'])
print_results(lambda task: task_metr[task], lambda task: task_max[task], ['dep', 'pos', 'ner'], langs,
            ['private-focused-0.75-dep-ner-pos-cs-es'], ['mt'])
print('ml')
print_results(lambda task: task_metr[task], lambda task: task_max[task], ['dep', 'pos', 'ner'], langs,
            ['no-adv-tsh-focused-dep-ner-pos-cs-es'], ['ml'])
print_results(lambda task: task_metr[task], lambda task: task_max[task], ['dep', 'pos', 'ner'], langs,
            ['private-focused-dep-ner-pos-cs-es'], ['ml'])
print_results(lambda task: task_metr[task], lambda task: task_max[task], ['dep', 'pos', 'ner'], langs,
            ['no-adv-tsh-focused-0.75-dep-ner-pos-cs-es'], ['ml'])
print_results(lambda task: task_metr[task], lambda task: task_max[task], ['dep', 'pos', 'ner'], langs,
            ['private-focused-0.75-dep-ner-pos-cs-es'], ['ml'])
print('mtml')
print_results(lambda task: task_metr[task], lambda task: task_max[task], ['dep', 'pos', 'ner'], langs,
            ['no-adv-tsh-focused-dep-ner-pos-cs-es'], ['mtml'])
print_results(lambda task: task_metr[task], lambda task: task_max[task], ['dep', 'pos', 'ner'], langs,
            ['private-focused-dep-ner-pos-cs-es'], ['mtml'])
print_results(lambda task: task_metr[task], lambda task: task_max[task], ['dep', 'pos', 'ner'], langs,
            ['no-adv-tsh-focused-0.75-dep-ner-pos-cs-es'], ['mtml'])
print_results(lambda task: task_metr[task], lambda task: task_max[task], ['dep', 'pos', 'ner'], langs,
            ['private-focused-0.75-dep-ner-pos-cs-es'], ['mtml'])





In [ ]:
draw_graphs(lambda task: task_metr[task], tasks, langs, 'test', ['vanilla'])

In [ ]:
draw_graphs(lambda task: task_metr[task], tasks, langs, 'test',
            run_codes=['vanilla', 'vanilla300', 'vanilla400'],
            run_types=['mtml'])